In [1]:
import pandas as pd
import hmac
import hashlib
import socket
import json

# HMAC verification function
def verify_hmac(row, secret_key):
    data = ', '.join(map(str, row.drop(labels=['HMAC']).values)).encode()
    computed_hmac = hmac.new(secret_key, data, hashlib.sha256).hexdigest()
    return computed_hmac == row['HMAC']

# UDP packet sender function
def send_udp_packets(df, destination_ip, destination_port):
    # Create a UDP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    
    # Counters for tracking sent packets
    valid_sent = 0
    invalid_blocked = 0

    for index, row in df.iterrows():
        packet = {
            "data": ', '.join(map(str, row.drop(labels=['Valid', 'HMAC']).values)),
            "HMAC": row['HMAC'],
            "Valid": row['Valid']
        }

        # Serialize packet to JSON
        packet_json = json.dumps(packet).encode()

        if row['Valid']:
            # Send packet
            sock.sendto(packet_json, (destination_ip, destination_port))

        # Update counters
        if row['Valid']:
            valid_sent += 1
        else:
            invalid_blocked += 1

        # Report every 1000 packets
        if valid_sent % 1000 == 0 and row['Valid']:
            print(f"{valid_sent} valid packets sent...")
        if invalid_blocked % 1000 == 0 and not row['Valid']:
            print(f"{invalid_blocked} invalid packets blocked...")   
    # Close socket
    sock.close()
    print("All packets sent.")
    print(f"Total valid packets sent: {valid_sent}")
    print(f"Total invalid packets blocked: {invalid_blocked}")

# Configuration
file_path = '/Users/erikatorres/cs 4371 hmac/benign_with_hmac.csv'
shared_key = b'thisisthekey'
destination_ip = '127.0.0.1'
destination_port = 8080      

# Read and process the CSV file
df = pd.read_csv(file_path)
df['Valid'] = df.apply(lambda row: verify_hmac(row, shared_key), axis=1)

# Send packets
send_udp_packets(df, destination_ip, destination_port)

1000 invalid packets blocked...
2000 invalid packets blocked...
3000 invalid packets blocked...
4000 invalid packets blocked...
5000 invalid packets blocked...
6000 invalid packets blocked...
1000 valid packets sent...
7000 invalid packets blocked...
8000 invalid packets blocked...
9000 invalid packets blocked...
10000 invalid packets blocked...
11000 invalid packets blocked...
12000 invalid packets blocked...
13000 invalid packets blocked...
2000 valid packets sent...
14000 invalid packets blocked...
3000 valid packets sent...
15000 invalid packets blocked...
16000 invalid packets blocked...
4000 valid packets sent...
5000 valid packets sent...
6000 valid packets sent...
7000 valid packets sent...
8000 valid packets sent...
9000 valid packets sent...
10000 valid packets sent...
11000 valid packets sent...
12000 valid packets sent...
17000 invalid packets blocked...
18000 invalid packets blocked...
13000 valid packets sent...
14000 valid packets sent...
19000 invalid packets blocked...